[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/act-cms/molecular-representations/blob/main/4-hybridisation.ipynb)


# Hybridisation Explorer: from **sp** to **sp³d²**

*General Chemistry & Cyberinfrastructure Skills Module*


### Warm‑Up Questions

**WQ‑1.** What is the **steric number** of an atom, and how does it relate to hybridisation? Give an example of an atom with steric number 4.

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

**WQ‑2.** Why do we need the concept of hybridisation? What problem does it solve that simple atomic orbitals cannot explain?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>


## Learning Objective
By the end of this activity, you should be able to **describe** orbital hybridisation and **determine** whether a particular atom is *sp*, *sp²*, *sp³*, *sp³d*, or *sp³d²* hybridised.


## Prerequisites
- Python ≥ 3.8
- **RDKit** for cheminformatics
- *(Optional)* **py3Dmol** for 3‑D visualisation

> **Tip:** If you’re running this on Google Colab, execute the installation cell below **first**.

In [ ]:
import sys
if 'google.colab' in sys.modules:
    # Install RDKit and py3Dmol if running in Google Colab
    !pip install rdkit py3Dmol -q

from rdkit import Chem
from rdkit.Chem import AllChem

# Helper function to convert RDKit HybridizationType to a nice string 
def hybrid_str(atom):
    return str(atom.GetHybridization()).split('.')[-1].lower()

# (Optional) quick viewer
try:
    import py3Dmol
    def show_3d(mol):
        m2 = Chem.AddHs(mol)
        # Use ETKDG for better 3D embedding, especially for complex geometries
        try:
            AllChem.EmbedMolecule(m2, AllChem.ETKDG(), randomSeed=0xf00d)
            # Skip UFF optimization for hypervalent molecules that UFF can't handle
            try:
                AllChem.UFFOptimizeMolecule(m2)
            except:
                pass  # Continue without optimization if UFF fails
        except:
            # Fallback to basic embedding if ETKDG fails
            AllChem.EmbedMolecule(m2, randomSeed=0xf00d)
        mb = Chem.MolToMolBlock(m2)
        v = py3Dmol.view()
        v.addModel(mb, 'mol')
        v.setStyle({'stick': {}})
        v.zoomTo()
        return v.show()
except ModuleNotFoundError:
    def show_3d(mol):
        print('🛈 Install py3Dmol for a 3‑D view.')


## Quick Concept Check 🔍
Hybridisation is a *model* that helps rationalise molecular shape and bonding.

| Hybridisation | Steric number* | Example geometry |
|---------------|---------------|------------------|
| sp            | 2             | Linear |
| sp²           | 3             | Trigonal planar |
| sp³           | 4             | Tetrahedral |
| sp³d          | 5             | Trigonal bipyramidal |
| sp³d²         | 6             | Octahedral |

*Steric number = *σ*‑bonds + lone‑pairs on the atom.

## Worked Example 1 — Methane
We’ll let RDKit calculate the hybridisation of each atom in methane (*CH₄*).

In [ ]:
methane = Chem.MolFromSmiles('C')
for atom in methane.GetAtoms():
    print(f'Atom {atom.GetIdx():>2} ({atom.GetSymbol():<2}) → {hybrid_str(atom)}')

# Visualise the 3‑D structure (optional)
show_3d(methane)

## Worked Example 2 — Ethene vs Acetylene
Ethene (*C₂H₄*) has a C=C double bond, whereas acetylene (*C₂H₂*) has a C≡C triple bond.

In [ ]:
for name, smiles in {'Ethene':'C=C', 'Acetylene':'C#C'}.items():
    mol = Chem.MolFromSmiles(smiles)
    hybs = {hybrid_str(a) for a in mol.GetAtoms() if a.GetSymbol() == 'C'}
    print(f'{name:10s}: carbon atoms → {", ".join(sorted(hybs))}')
    show_3d(mol)


## Worked Example 3 — Beyond Octet
Let’s inspect *PCl₅* (trigonal bipyramidal, sp³d) and *SF₆* (octahedral, sp³d²).

In [ ]:
examples = {
    'Phosphorus pentachloride': 'ClP(Cl)(Cl)(Cl)Cl',
    'Sulfur hexafluoride'     : 'FS(F)(F)(F)(F)F'
}

for name, smi in examples.items():
    mol = Chem.MolFromSmiles(smi)
    # central atom is the first heavy atom RDKit parses
    central = max(
        (a for a in mol.GetAtoms() if a.GetSymbol() not in ['Cl', 'F']),
        key=lambda a: a.GetTotalDegree()
    )
    print(f'{name}: central {central.GetSymbol()} → {hybrid_str(central)}')
    show_3d(mol)

<span style="color:red"><strong>Note:</strong> The 3D geometry shown for SF<sub>6</sub> above is incorrect because RDKit’s default force field does not handle hypervalent (expanded octet) sulfur compounds correctly. The bond angles or geometry may not match the expected octahedral structure.</span>


## Your Turn 📝
Pick **two** molecules you’re curious about (e.g. *CO₂*, *NH₃*, *XeF₂*…).
1. Create an RDKit molecule from SMILES.
2. Write a loop that prints the hybridisation of **every** atom.
3. Compare the results with VSEPR predictions.

*(💡 Need a hint? Revisit the `for`‑loop in the methane example.)*

In [ ]:
# TODO 1: Replace with your own SMILES strings
my_smiles = ['[C-]#[O+]', 'N']  # ← EDIT ME

for smi in my_smiles:
    mol = Chem.MolFromSmiles(smi)
    print(f"SMILES: {smi}")
    for atom in mol.GetAtoms():
        print(f"  Atom {atom.GetIdx()} ({atom.GetSymbol()}): {hybrid_str(atom)}")
    show_3d(mol)
    print()

### Critical‑Thinking Questions

**CTQ‑1.** Why might an atom with **sp³d²** hybridisation be able to form more bonds than one with **sp³** hybridisation? What does this tell us about the relationship between hybridisation and molecular geometry?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

**CTQ‑2.** In what situations might the hybridisation model **fail** to accurately predict molecular geometry? Give an example of a molecule where hybridisation theory might not work as expected.

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>


### Challenge: Write a Helper Function
Write a function `predict_hybridisation(atom)` that returns the hybridisation based **only** on the steric number (σ bonds + lone pairs) *without* using RDKit’s built‑in method. Test it on a few molecules.

In [ ]:
def predict_hybridisation(atom):
    """Return a string like 'sp3' based on steric number.*
    *Assumes atom has already computed valence & lone pair info.*"""
    pass


## Summary & Next Steps
- **Hybridisation** links atomic orbital mixing to molecular geometry.
- RDKit can programmatically report each atom’s hybridisation, enabling rapid checks across many structures.
- Challenge yourself to automate VSEPR‑based predictions **without** RDKit’s helper!

> **Extension:** Combine this workflow with data from databases (e.g. the *Materials Project* API) to analyse hybridisation trends across thousands of molecules.